# Collectl CPU Log Analysis

## Functionalities
- Plot CPU utilization graphs.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    collectl.tar.gz
  ...
  [node-n]/
    collectl.tar.gz
```
A tarball `collectl.tar.gz` contains log files. The log file extension identifies the type of resource monitored:
- `.cpu.gz`: CPU monitoring log file.
- `.numa.gz`: memory monitoring log file.
- `.dsk.gz`: disk I/O monitoring log file.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_[TIMESTAMP]"
# Ramp up duration (in sec)
RAMP_UP_DURATION = 180
# Ramp down duration (in sec)
RAMP_DOWN_DURATION = 180

########## CPU
# Analyzed metric (options: "user", "nice", "system", "wait", "irq", "soft",
# "steal", "idle", "total", "guest", "guest_n", "intrpt")
COLLECTL_CPU_METRIC = "total"
# Filter CPU cores
COLLECTL_CPU_CORES = None

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from utils.utils import *

experiment_dirpath = os.path.join(os.path.abspath(""), "..", "data", EXPERIMENT_DIRNAME)

## Log Parsing & Processing

In [ ]:
# Build data frame
cpu = pd.concat([df[2] for df in get_collectl_cpu_df(experiment_dirpath)])

In [ ]:
# Filter data frames
if COLLECTL_CPU_CORES:
    cpu = cpu[(cpu["hw_no"].isin(COLLECTL_CPU_CORES))]

In [ ]:
# Extract experiment information
start_time = get_experiment_start_time(experiment_dirpath)
node_names = get_node_names(experiment_dirpath)

In [ ]:
# (Re) Build columns
cpu["timestamp"] = cpu.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
cpu["window"] = cpu.apply(lambda r: int(r["timestamp"]), axis=1)

In [ ]:
# (Re) Create index
cpu.set_index("timestamp", inplace=True)
cpu.sort_index(inplace=True)

## CPU Monitoring

In [ ]:
# Plot CPU utilization (1-sec granularity)
fig = plt.figure(figsize=(24, len(node_names) * 12))
for (i, node_name) in enumerate(node_names):
    df = cpu[(cpu["node_name"] == node_name)]
    df = df.groupby(["window"])[COLLECTL_CPU_METRIC].mean()
    ax = fig.add_subplot(len(node_names), 1, i + 1)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.set_ylim((0, 100))
    ax.grid(alpha=0.75)
    df.plot(ax=ax, kind="line", title="%s - CPU Utilization" % node_name, xlabel="Time (seconds)",
            ylabel="%s (%%)" % COLLECTL_CPU_METRIC, grid=True, legend=False, yticks=range(0, 101, 10))

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot CPU utilization (millisec granularity)
if MIN_TIME and MAX_TIME:
    fig = plt.figure(figsize=(24, len(node_names) * 12))
    for (i, node_name) in enumerate(node_names):
        df = cpu[(cpu["node_name"] == node_name)]
        df = df[(df.index >= MIN_TIME) & (df.index <= MAX_TIME)]
        df = df.groupby(["timestamp", "hw_no"])[COLLECTL_CPU_METRIC].mean()
        df = df.unstack()
        ax = fig.add_subplot(len(node_names), 1, i + 1)
        ax.set_xlim((df.index.min(), df.index.max()))
        ax.set_ylim((0, 100))
        ax.grid(alpha=0.75)
        df.plot(ax=ax, kind="line", title="%s - CPU Utilization" % node_name, xlabel="Time (seconds)",
                ylabel="%s (%%)" % COLLECTL_CPU_METRIC, grid=True, legend=True, yticks=range(0, 101, 10))